# XGBoost Regressor

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# fix_yahoo_finance is used to fetch data 
import fix_yahoo_finance as yf
yf.pdr_override()

In [2]:
# input
symbol = 'AMD'
start = '2014-01-01'
end = '2019-01-01'

# Read data 
dataset = yf.download(symbol,start,end)

# View Columns
dataset.head()

[*********************100%***********************]  1 of 1 downloaded


Open  High   Low  Close  Adj Close    Volume
Date                                                    
2014-01-02  3.85  3.98  3.84   3.95       3.95  20548400
2014-01-03  3.98  4.00  3.88   4.00       4.00  22887200
2014-01-06  4.01  4.18  3.99   4.13       4.13  42398300
2014-01-07  4.19  4.25  4.11   4.18       4.18  42932100
2014-01-08  4.23  4.26  4.14   4.18       4.18  30678700

In [3]:
# Create more data
dataset['Increase/Decrease'] = np.where(dataset['Volume'].shift(-1) > dataset['Volume'],1,0)
dataset['Buy_Sell_on_Open'] = np.where(dataset['Open'].shift(-1) > dataset['Open'],1,-1)
dataset['Buy_Sell'] = np.where(dataset['Adj Close'].shift(-1) > dataset['Adj Close'],1,-1)
dataset['Return'] = dataset['Adj Close'].pct_change()
dataset = dataset.dropna()
dataset.head()

Open  High   Low  Close  Adj Close    Volume  Increase/Decrease  \
Date                                                                          
2014-01-03  3.98  4.00  3.88   4.00       4.00  22887200                  1   
2014-01-06  4.01  4.18  3.99   4.13       4.13  42398300                  1   
2014-01-07  4.19  4.25  4.11   4.18       4.18  42932100                  0   
2014-01-08  4.23  4.26  4.14   4.18       4.18  30678700                  0   
2014-01-09  4.20  4.23  4.05   4.09       4.09  30667600                  0   

            Buy_Sell_on_Open  Buy_Sell    Return  
Date                                              
2014-01-03                 1         1  0.012658  
2014-01-06                 1         1  0.032500  
2014-01-07                 1        -1  0.012107  
2014-01-08                -1        -1  0.000000  
2014-01-09                -1         1 -0.021531

In [4]:
dataset.shape

(1257, 10)

In [5]:
X = dataset[['Open', 'High', 'Low', 'Volume']].values
y = dataset['Adj Close'].values

In [6]:
from xgboost import XGBRegressor

# XGboost algorithm
xgb = XGBRegressor(max_depth=5, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
xgb.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.1, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=2000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [8]:
y_pred = xgb.predict(X_test)

In [9]:
from sklearn.metrics import mean_squared_error
from sklearn import metrics

print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

The rmse of prediction is: 0.605212187519


In [10]:
print('XGBoost Regression Score:', xgb.score(X_test, y_test))

XGBoost Regression Score: 0.990494075323
